## 번개장터 데이터 수집을 위한 크롤러 (최종본)

- 번개장터 데이터를 수집하기 위한 크롤러 입니다.
- 크롤링하는 과정을 함수화(모듈화)하여 편의성을 높입니다.
- 크롤링 후에는 중복 제거, 날짜컬럼 생성의 과정을 거칩니다.
- 키워드 리스트를 만들어 for문을 수행합니다.

### import package

In [20]:
import requests # 크롤링을 위함
import json # json파일을 다루기위함
import time # 크롤링에 딜레이를 두기 위함
from datetime import datetime, timedelta # 날짜데이터 변환을 위함

import pandas as pd
import numpy as np

# 데이터프레임이 짤려보이는 것을 해결해줌
pd.options.display.max_rows=100
pd.options.display.max_columns=100

### Crawler

- keyword와 url, n을 변수로 받아 크롤링을 진행한다.
- keyword와 url은 리스트이고, n은 1페이지부터 n페이지까지 크롤링해준다. (**default=20** : 20페이지 * 100 = 2000개)
- 데이터프레임을 반환받으며, 크롤러 함수를 변수에 선언해주어야한다.

In [21]:
def bunkae_crawler(keyword, n=20):
    df = pd.DataFrame()
    
    for i in range(0,n):

        # 차단막는 코드, 랜덤으로 time.sleep 지정
        seed = np.random.randint(100)
        np.random.seed(seed)
        a = np.random.randint(3)
        time.sleep(a)
        
        url_formating = url.format(keyword, i) # url에 포맷팅을 적용한다.
        info = {
        'referer': 'https://m.bunjang.co.kr/',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36'
        } # info는 변화없음

        print(f'{i}th, {(i+1)*100}개째 상품을 크롤링 중')
        # requests로 데이터 요청하기
        resp = requests.get(url_formating, headers = info)

        if resp.status_code == requests.codes.ok:
            data = json.loads(resp.text)
            next_df = pd.DataFrame(data['list'])
            df = pd.concat([df, next_df])

        else:
            print(f'{i}번째 요청이 잘못되었습니다.')
            pass # 넘겨준다.

    print('크롤링 완료!!')
    result = df.reset_index(drop=True)
    return result

### Deduplicator

- 크롤링 과정에서 상품이 등록되면 한칸씩 밀리면서 같은 상품이 두번 크롤링되는 경우가 있다. 이를 해결하기 위한 함수이다.
- pid(상품명)를 기준으로 중복을 제거한다. (상품명은 unique id)

In [22]:
def deduplicator(df):
    df = df.drop_duplicates(['pid'], keep='first')
    return df

### Datetime maker

- update_time 컬럼 : 1970-01-01 09:00 이후로 초를 카운트한 값이다.<br>
    - 1970-01-01 09:00라는 기준에서 5분 이내의 오차가 존재한다. (큰 문제 없음)
- update_time 컬럼을 datetime과 timedelta를 이용해 날짜로 변환한다.
- 새로 생성된 컬럼<br>
    - datetime : timedelta형식의 날짜데이터

In [23]:
def get_date(x):
    return datetime(1970,1,1,9) + timedelta(seconds=int(x))

In [24]:
def datetime_maker(df):
    df['datetime'] = df['update_time'].apply(get_date)
    return df

# Final Keywords Crawling

- 위의 함수들을 모두 적용한 셀이며, 이것만 실행하면 keywords 모두 다 크롤링해온다.
- 키워드 예시 : '뉴발란스 992', '조던1 하이', '나이키 덩크로우'

**[사용법]**
1. keywords 리스트에 검색어를 차례로 입력한다.
2. url은 keyword에 해당되는 Request URL을 입력하였고, 이는 키워드가 달라져도 변함이 없다.<br>
    1. url은 개발자도구 Network 탭에서 페이지클릭 후 find_v2.json파일에 해당되는 Request URL을 가져온다.
    2. find_v2.json?q={} => 상품명을 {}로 변환한다 (함수 내에서 포맷팅을 위해)
    3. order=date&page={} => 페이지를 {}로 변환한다 (함수 내에서 포맷팅을 위해)
    4. 검색 시점에 해당되는 request_id가 있으나 시간데이터로 모두 같은 url을 사용함으로써 통일시킨다. (2021-11-11기준)

In [25]:
# keywords = ['나이키 덩크로우 골든로드', '나이키 덩크로우 라이트본','나이키 덩크로우 바시티그린', '나이키 덩크로우 범고래','나이키 덩크로우 유니버시티블루',
#  '나이키 덩크로우 코스트','나이키 덩크하이 네이비','나이키 덩크하이 범고래','나이키 덩크하이 오렌지','뉴발란스 992 그레이','뉴발란스 992 네이비',
#  '뉴발란스 992 블랙그레이','뉴발란스 992 화이트실버','뉴발란스 993 그레이','뉴발란스 993 네이비','뉴발란스 993 블랙','조던 1 로우 스타피쉬',
#  '조던 1 로우 울프그레이','조던 1 로우 트레비스 스캇''조던1 미드 그레이포그','조던1 미드 스모크그레이','조던1 미드 울프그레이',
# '조던1 미드 짐레드','조던1 하이 다크모카','조던1 하이 스모크그레이', '조던1 하이 하이퍼로얄']

In [30]:
keywords = ['조던1 로우']
url = 'https://api.bunjang.co.kr/api/1/find_v2.json?q={}&order=date&page={}&request_id=20211110194853&stat_uid=77848616&token=c63b703f121940d189c222b3335d80ed&stat_device=w&n=100&stat_category_required=1&req_ref=search&version=4'

In [31]:
# test로 keyword별로 500개씩만 가져와본다. (n=5 : default는 2000개씩)

bunkae = pd.DataFrame()
for keyword in keywords:
    print(f'{keyword} 크롤링 시작!')
    shoe_df = bunkae_crawler(keyword, n=60)
    bunkae = pd.concat([bunkae, shoe_df])
    print('----'*10)
print('모든 신발 크롤링 완료!')
bunkae = bunkae.reset_index(drop=True)
bunkae_df = bunkae.copy()
deduplicator(bunkae_df)
bunkae_df = datetime_maker(bunkae_df)

조던1 로우 크롤링 시작!
0th, 100개째 상품을 크롤링 중
1th, 200개째 상품을 크롤링 중
2th, 300개째 상품을 크롤링 중
3th, 400개째 상품을 크롤링 중
4th, 500개째 상품을 크롤링 중
5th, 600개째 상품을 크롤링 중
6th, 700개째 상품을 크롤링 중
7th, 800개째 상품을 크롤링 중
8th, 900개째 상품을 크롤링 중
9th, 1000개째 상품을 크롤링 중
10th, 1100개째 상품을 크롤링 중
11th, 1200개째 상품을 크롤링 중
12th, 1300개째 상품을 크롤링 중
13th, 1400개째 상품을 크롤링 중
14th, 1500개째 상품을 크롤링 중
15th, 1600개째 상품을 크롤링 중
16th, 1700개째 상품을 크롤링 중
17th, 1800개째 상품을 크롤링 중
18th, 1900개째 상품을 크롤링 중
19th, 2000개째 상품을 크롤링 중
20th, 2100개째 상품을 크롤링 중
21th, 2200개째 상품을 크롤링 중
22th, 2300개째 상품을 크롤링 중
23th, 2400개째 상품을 크롤링 중
24th, 2500개째 상품을 크롤링 중
25th, 2600개째 상품을 크롤링 중
26th, 2700개째 상품을 크롤링 중
27th, 2800개째 상품을 크롤링 중
28th, 2900개째 상품을 크롤링 중
29th, 3000개째 상품을 크롤링 중
30th, 3100개째 상품을 크롤링 중
31th, 3200개째 상품을 크롤링 중
32th, 3300개째 상품을 크롤링 중
33th, 3400개째 상품을 크롤링 중
34th, 3500개째 상품을 크롤링 중
35th, 3600개째 상품을 크롤링 중
36th, 3700개째 상품을 크롤링 중
37th, 3800개째 상품을 크롤링 중
38th, 3900개째 상품을 크롤링 중
39th, 4000개째 상품을 크롤링 중
40th, 4100개째 상품을 크롤링 중
41th, 4200개째 상품을 크롤링 중
42th, 4300개째 상품을 크롤링 중
43th, 4400개째 상품

In [33]:
bunkae_df['name'].head()

0       조던1 로우 gs 라이트 스모크 그레이 240
1            나이키 에어조던1 로우 블랙토 260
2        나이키 조던1 로우 다크 파우더 블루 270
3             나이키 조던1  로우 se 크레이터
4    에어디올 로우 41/ 260 새상품 한정판 가격내림
Name: name, dtype: object

**Raw Data 수집 완료**

### Data Save

In [32]:
bunkae_df.to_csv('./data/jodan1_low.csv')